In [6]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips
import torch
from torchmetrics.image import VisualInformationFidelity
from torchmetrics.image.fid import FrechetInceptionDistance

DEVICE = 'cuda:1'

# Define the function to compute MAE
def compute_mae(image1, image2):
    return np.abs(image1 - image2).mean()

def compute_mape(image1, image2):
    return np.abs((image1 - image2) / (image1 + 1e-6)).mean() * 100

loss_fn_vgg = lpips.LPIPS(net='vgg').to(DEVICE)
vif = VisualInformationFidelity().to(DEVICE)
# fid = FrechetInceptionDistance(normalize=True, feature=2048).to(DEVICE)

# Paths to the folders
gt_folder = "/home/PET-CT/splited_data_15k/test/B"
# pre_folder = "results/108_CT2PET_UncerBBDM3c/LBBDM-f4/sample_to_eval/200"
pre_folder = "/home/PET-CT/thaind/BBDM_folk/results/improved_sampling_BBDM_CT_s=0.3/LBBDM-f4/sample_to_eval/200"
# Lists to store the computed metrics for each pair
ssim_scores = []
psnr_scores = []
mae_scores = []
mape_scores = []
lpips_scores = []
vif_scores = []
pre_dist = []
gt_dist = []

# Iterate through the files in the ground truth folder
for filename in os.listdir(gt_folder):
    # Make sure the file is a numpy array
    if filename.endswith(".npy"):
        # Construct the paths for the corresponding ground truth and predicted files
        try:
            gt_path = os.path.join(gt_folder, filename)
            pre_path = os.path.join(pre_folder, filename)
        
            # Load the images as numpy arrays
            gt_img = np.load(gt_path, allow_pickle=True)
            pre_img = np.load(pre_path, allow_pickle=True)
        except:
            continue   
        
        # Preprocess the predicted image
        pre_img = pre_img.mean(axis=-1) / 32767.0
        
        # Normalize the ground truth image
        # gt_img = gt_img.mean(axis=-1) / 32767.0
        gt_img = gt_img / 32767.0
        
        pre_img = pre_img.astype(np.float32)
        gt_img = gt_img.astype(np.float32)
        
        # Calculate the SSIM, PSNR, and MAE for this pair
        ssim_score = ssim(pre_img, gt_img, data_range=1)
        psnr_score = psnr(pre_img, gt_img, data_range=1)
        mae = compute_mae(pre_img, gt_img)
        mape = compute_mape(1-gt_img, 1-pre_img)

        pre_img = torch.from_numpy(pre_img)
        pre_img = (pre_img - 0.5) * 2
        pre_img = pre_img.unsqueeze(0)
        pre_img = pre_img.expand(3, -1, -1)
        pre_img = pre_img.unsqueeze(0).to(DEVICE)
        
        gt_img = torch.from_numpy(gt_img)
        gt_img = (gt_img - 0.5) * 2
        gt_img = gt_img.unsqueeze(0)
        gt_img = gt_img.expand(3, -1, -1)
        gt_img = gt_img.unsqueeze(0).to(DEVICE)

        lpips_score = loss_fn_vgg(pre_img, gt_img) 
        lpips_score = lpips_score.detach().cpu().numpy()[0, 0, 0, 0]
        
        vif_score = vif(pre_img, gt_img).detach().cpu().numpy()
        
        # Append the scores to the corresponding lists
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        mae_scores.append(mae * 32767)
        mape_scores.append(mape)
        lpips_scores.append(lpips_score)
        vif_scores.append(vif_score)
        # pre_dist.append(pre_img.detach().cpu())
        # gt_dist.append(gt_img.detach().cpu())

# Calculate the mean scores over all pairs
mean_ssim = np.mean(ssim_scores)
mean_psnr = np.mean(psnr_scores)
mean_mae = np.mean(mae_scores)
mean_mape = np.mean(mape_scores)
mean_lpips = np.mean(lpips_scores)
mean_vif = np.mean(vif_scores)

# pre_dist = torch.cat(pre_dist, dim=0).to(DEVICE)
# gt_dist = torch.cat(gt_dist, dim=0).to(DEVICE)

# fid.update(gt_dist, real=True)
# fid.update(pre_dist, real=False)
# fid_score = fid.compute().detach().cpu().numpy()

# Print the mean metrics
print("Mean SSIM: {}".format(mean_ssim))
print("Mean PSNR: {}".format(mean_psnr))
print("Mean MAE: {}".format(mean_mae))
print("Mean MAPE: {}".format(mean_mape))
print("Mean LPIPS: {}".format(mean_lpips))
print("Mean VIF: {}".format(mean_vif))
# print("FID: {}".format(fid_score))

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/root/anaconda3/envs/BBDM/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/BBDM/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /root/anaconda3/envs/BBDM/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth
Mean SSIM: 0.936423588162284
Mean PSNR: 26.996204820036535
Mean MAE: 373.42105203404884
Mean MAPE: 425.3185685410785
Mean LPIPS: 0.047323718667030334
Mean VIF: 0.91739422082901


In [2]:
import os
import numpy as np

TEST_DIR = '/home/PET-CT/splited_data_15k/test/B'
STATIC_THRESH_HOLD = 100

# Khởi tạo biến để lưu trữ tọa độ tối thiểu và tối đa
min_x, min_y = np.inf, np.inf
max_x, max_y = -np.inf, -np.inf

for filename in os.listdir(TEST_DIR):
    if filename.endswith(".npy"):
        try:
            pet_path = os.path.join(TEST_DIR, filename)
            pet_img = np.load(pet_path, allow_pickle=True)
        except:
            continue   
    
        pet_img = pet_img / 32767. * 255.
        
        # Tìm các pixel thỏa mãn điều kiện
        y_indices, x_indices = np.where(pet_img >= STATIC_THRESH_HOLD)
        
        if len(x_indices) == 0 or len(y_indices) == 0:
            # Không có pixel nào thỏa mãn điều kiện
            continue
        
        # Cập nhật tọa độ tối thiểu và tối đa
        min_x = min(min_x, x_indices.min())
        max_x = max(max_x, x_indices.max())
        min_y = min(min_y, y_indices.min())
        max_y = max(max_y, y_indices.max())

# Kiểm tra nếu không tìm thấy pixel nào thỏa mãn
if min_x == np.inf:
    print("Không tìm thấy pixel nào thỏa mãn điều kiện.")
else:
    # In ra tọa độ của vùng chữ nhật bao quanh
    print(f"Vùng chữ nhật bao phủ: (({min_y}, {min_x}), ({max_y}, {max_x}))")

Vùng chữ nhật bao phủ: ((54, 38), (175, 232))


In [3]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips
import torch
from torchmetrics.image import VisualInformationFidelity
from torchmetrics.image.fid import FrechetInceptionDistance

DEVICE = 'cuda:1'

# Define the function to compute MAE
def compute_mae(image1, image2):
    return np.abs(image1 - image2).mean()

def compute_mape(image1, image2):
    return np.abs((image1 - image2) / (image1 + 1e-6)).mean() * 100

loss_fn_vgg = lpips.LPIPS(net='vgg').to(DEVICE)
vif = VisualInformationFidelity().to(DEVICE)
# fid = FrechetInceptionDistance(normalize=True, feature=2048).to(DEVICE)

# Paths to the folders
gt_folder = "/home/PET-CT/splited_data_15k/test/B"
# pre_folder = "results/108_CT2PET_UncerBBDM3c/LBBDM-f4/sample_to_eval/200"
pre_folder = "/home/PET-CT/thaind/BBDM_folk/results/45_CPDM_gaussian_noise_blur_segment_dropout_0.3_seed1234/LBBDM-f4/sample_to_eval/200"
# Lists to store the computed metrics for each pair
ssim_scores = []
psnr_scores = []
mae_scores = []
mape_scores = []
lpips_scores = []
vif_scores = []
pre_dist = []
gt_dist = []

# Iterate through the files in the ground truth folder
for filename in os.listdir(gt_folder):
    # Make sure the file is a numpy array
    if filename.endswith(".npy"):
        # Construct the paths for the corresponding ground truth and predicted files
        try:
            gt_path = os.path.join(gt_folder, filename)
            pre_path = os.path.join(pre_folder, filename)
        
            # Load the images as numpy arrays
            gt_img = np.load(gt_path, allow_pickle=True)
            pre_img = np.load(pre_path, allow_pickle=True)
        except:
            continue   
        
        # Preprocess the predicted image
        pre_img = pre_img.mean(axis=-1) / 32767.0
        
        # Normalize the ground truth image
        # gt_img = gt_img.mean(axis=-1) / 32767.0
        gt_img = gt_img / 32767.0
        
        pre_img = pre_img.astype(np.float32)
        gt_img = gt_img.astype(np.float32)
        
        pre_img = pre_img[min_y:max_y+1, min_x:max_x+1]
        gt_img = gt_img[min_y:max_y+1, min_x:max_x+1]
        
        # Calculate the SSIM, PSNR, and MAE for this pair
        ssim_score = ssim(pre_img, gt_img, data_range=1)
        psnr_score = psnr(pre_img, gt_img, data_range=1)
        mae = compute_mae(pre_img, gt_img)
        mape = compute_mape(1-gt_img, 1-pre_img)

        pre_img = torch.from_numpy(pre_img)
        pre_img = (pre_img - 0.5) * 2
        pre_img = pre_img.unsqueeze(0)
        pre_img = pre_img.expand(3, -1, -1)
        pre_img = pre_img.unsqueeze(0).to(DEVICE)
        
        gt_img = torch.from_numpy(gt_img)
        gt_img = (gt_img - 0.5) * 2
        gt_img = gt_img.unsqueeze(0)
        gt_img = gt_img.expand(3, -1, -1)
        gt_img = gt_img.unsqueeze(0).to(DEVICE)

        lpips_score = loss_fn_vgg(pre_img, gt_img) 
        lpips_score = lpips_score.detach().cpu().numpy()[0, 0, 0, 0]
        
        vif_score = vif(pre_img, gt_img).detach().cpu().numpy()
        
        # Append the scores to the corresponding lists
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        mae_scores.append(mae * 32767)
        mape_scores.append(mape)
        lpips_scores.append(lpips_score)
        vif_scores.append(vif_score)
        # pre_dist.append(pre_img.detach().cpu())
        # gt_dist.append(gt_img.detach().cpu())

# Calculate the mean scores over all pairs
mean_ssim = np.mean(ssim_scores)
mean_psnr = np.mean(psnr_scores)
mean_mae = np.mean(mae_scores)
mean_mape = np.mean(mape_scores)
mean_lpips = np.mean(lpips_scores)
mean_vif = np.mean(vif_scores)

# pre_dist = torch.cat(pre_dist, dim=0).to(DEVICE)
# gt_dist = torch.cat(gt_dist, dim=0).to(DEVICE)

# fid.update(gt_dist, real=True)
# fid.update(pre_dist, real=False)
# fid_score = fid.compute().detach().cpu().numpy()

# Print the mean metrics
print("Mean SSIM: {}".format(mean_ssim))
print("Mean PSNR: {}".format(mean_psnr))
print("Mean MAE: {}".format(mean_mae))
print("Mean MAPE: {}".format(mean_mape))
print("Mean LPIPS: {}".format(mean_lpips))
print("Mean VIF: {}".format(mean_vif))
# print("FID: {}".format(fid_score))

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /root/anaconda3/envs/BBDM/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth
Mean SSIM: 0.8307461762394196
Mean PSNR: 27.52068070489335
Mean MAE: 586.0668520037337
Mean MAPE: 1494.8422996787936
Mean LPIPS: 0.12800467014312744
Mean VIF: 0.5157314538955688
